This is the Python code needed to load a cooking dataset, add your custom structural tokens, tokenize the data, and prepare it for fine-tuning GPT-2 in Colab.

1. Install Libraries and Load Tokenizer 📦
Start by ensuring you have the necessary libraries and load the standard GPT-2 tokenizer.

In [ ]:
!pip install transformers datasets torch

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
import torch

# 1. Load the Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# Set the padding token to be the same as the end-of-sequence token (standard for GPT-style models)
tokenizer.pad_token = tokenizer.eos_token

2. Add Custom Special Tokens 🔖
To teach the model the structure of a recipe, you must add special tokens for the Title, Ingredients, Instructions, and the end of the recipe.

In [ ]:
# 2. Define and Add Custom Special Tokens
special_tokens_dict = {
    'additional_special_tokens': [
        '<|title|>', 
        '<|ingredients|>', 
        '<|instructions|>', 
        '<|endofrecipe|>'
    ]
}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

# You MUST resize the model's token embeddings to account for the new tokens
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer)) 
print(f"Added {num_added_toks} new tokens to the vocabulary.")

3. Load, Format, and Tokenize the Dataset 🍚
For this example, we'll use a placeholder for a Hugging Face dataset, and define the formatting logic. You will need to replace the data loading line with your chosen cooking dataset.

In [ ]:
# 3. Load a Cooking Dataset (Example: Using a small text file you create/upload)
# If your data is in a file named 'recipes.txt', use the following:
# raw_datasets = load_dataset('text', data_files={'train': 'recipes.txt'})

# If using a Hugging Face dataset (e.g., a simple recipes collection):
# **Replace 'corbt/all-recipes' with your chosen dataset name if needed.**
# Ensure the dataset has a 'text' column or adapt the code below.
raw_datasets = load_dataset('corbt/all-recipes', split='train[:1%]') # Using a small slice for quick testing

# Filter out empty entries if the dataset is large and messy
raw_datasets = raw_datasets.filter(lambda x: x['text'] is not None and len(x['text'].strip()) > 0)


# 4. Define the Tokenization and Formatting Function
# This function assumes each entry (row) in the dataset is one complete, formatted recipe string.
def tokenize_and_format(examples):
    # This is the core data preparation step: 
    # The tokenizer handles the text-to-ID conversion, truncation, and padding.
    return tokenizer(
        examples['text'], 
        truncation=True, 
        max_length=512, # Maximum context length for GPT-2
        padding='max_length'
    )

# 5. Apply the function to the dataset
tokenized_datasets = raw_datasets.map(tokenize_and_format, batched=True, remove_columns=["text"])

# 6. Create the Train/Validation Split
split_datasets = tokenized_datasets.train_test_split(test_size=0.1)
train_dataset = split_datasets['train']
eval_dataset = split_datasets['test']

4. Initialize Trainer and Train 🚀
Now you can use the properly defined train_dataset and eval_dataset variables.

In [ ]:
# 7. Setup Data Collator for Causal Language Modeling (CLM)
# mlm=False tells the collator this is for next-token prediction (GPT-2 style).
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 8. Set up Training Arguments
training_args = TrainingArguments(
    output_dir="./recipe_results",
    num_train_epochs=3,
    per_device_train_batch_size=4, # Adjust based on your Colab GPU memory
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./recipe_logs',
    logging_steps=50,
    evaluation_strategy="epoch",
)

# 9. Initialize and Start Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

# trainer.train() 
# Uncomment the line above to start the fine-tuning process!